In [291]:
import math

import cv2
import numpy as np
from skimage.metrics import peak_signal_noise_ratio, structural_similarity

In [292]:
def dpcm(img, bit, b=2.0):
    h, w = img.shape
    img_rec = np.zeros_like(img, dtype=np.float32)
    img_err = np.zeros_like(img)
    for i in range(h):
        for j in range(w):
            if j == 0:
                img_rec[i][j] = img[i][j]
                img_err[i][j] = 0.0
            else:
                p = img[i][j] - img_rec[i][j - 1]
                p2 = p / b + 128.0
                img_err[i][j] = p2
                p3 = p2 * b
                img_rec[i][j] = img_rec[i][j - 1] + p3
    print(ssim(img, img_rec))
    pnsr(img, img_rec)
    # SSIM = 0 6.3642775051914935  4.119348493168026
    # cv2.imwrite('../worker5/result/result_cz_{}.bmp'.format(bit), img_err)
    # cv2.imwrite('../worker5/result/result_yc_{}.bmp'.format(bit), img_rec)

In [293]:
def pnsr(img1, img2):
    h, w = img.shape
    mse = 0.0
    for i in range(h):
        for j in range(w):
            mse += math.pow((img1[i][j] - img2[i][j]), 2)
    mse = mse / (w * h)
    pn = 10 * math.log10(255.0 * 255.0 / mse)
    print(pn)


In [294]:
def ssim(img1, img2):
    C1 = (0.01 * 255) ** 2
    C2 = (0.03 * 255) ** 2
    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    kernel = cv2.getGaussianKernel(11, 1.5)
    window = np.outer(kernel, kernel.transpose())
    mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]  # valid
    mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
    mu1_sq = mu1 ** 2
    mu2_sq = mu2 ** 2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = cv2.filter2D(img1 ** 2, -1, window)[5:-5, 5:-5] - mu1_sq
    sigma2_sq = cv2.filter2D(img2 ** 2, -1, window)[5:-5, 5:-5] - mu2_sq
    sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2
    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) *
                                                            (sigma1_sq + sigma2_sq + C2))
    return ssim_map.mean()

In [295]:
if __name__ == '__main__':
    img = cv2.imread('result/menna_gray.bmp', cv2.IMREAD_GRAYSCALE)
    dpcm(img, 1, b=256)

0.0025954989814665577
-42.15642589797251
